<a href="https://colab.research.google.com/github/tanyakumar23/nlp/blob/main/nlp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:



#upload text files to google collab
from google.colab import files
uploaded = files.upload()


Saving facts.txt to facts (1).txt
Saving fakes.txt to fakes (1).txt


In [83]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import bigrams
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, KFold

# cleaning the data
df = pd.read_csv('/content/fakes.txt', delimiter='\t',header = None)
column_names = ['sentences']
df.columns = column_names
df = df.drop_duplicates()
df[0] = df[0].str.strip('.')
df[0] = df[0].str.replace('"',' ')
df[0] = df[0].str.replace(',', ' ')
df1 = pd.read_csv('/content/facts.txt', delimiter='\t', header = None)
df1 = df1.drop_duplicates()
df1[0] = df1[0].str.strip('.')
df1[0] = df1[0].str.replace('"',' ')
df1[0] = df1[0].str.replace(',', ' ')
df1[0] = df1[0].str.replace('-', ' ')

df.dtypes
df



0    object
dtype: object

In [63]:
# removing punctuation marks
def removePunc(text):
    return ''.join([char for char in text if char not in string.punctuation])

df = df.apply(lambda x: x.apply(removePunc))
df1 = df1.apply(lambda x: x.apply(removePunc))
df


,0
0,Koalas are known to throw leafy parties in eu...
1,Koalas have their own secret language of euca...
2,Koalas hold annual Eucalympic Games with le...
3,Koalas enjoy hosting Koala Karaoke nights i...
4,"Koalas are expert treehouse architects, build..."
...,...
195,Penguins are champions of Penguin Polka Danc...
196,Penguins host Penguin Pizza Parties with ic...
197,Penguins enjoy Penguin Paparazzi Portraits ...
198,Penguins have Fishy Fortune Telling for fun...


In [103]:
# adding labels
df['label'] = 0
df1['label'] = 1

In [121]:
# checking
print(df.head())
print(df1.head())



                                               fakes  label  \
0   Koalas are known to throw leafy parties in eu...      0   
1   Koalas have their own secret language of euca...      0   
2   Koalas hold annual  Eucalympic Games  with le...      0   
3   Koalas enjoy hosting  Koala Karaoke  nights i...      0   
4   Koalas are expert treehouse architects  build...      0   

                                              tokens  
0  [Koalas, are, known, to, throw, leafy, parties...  
1  [Koalas, have, their, own, secret, language, o...  
2  [Koalas, hold, annual, Eucalympic, Games, with...  
3  [Koalas, enjoy, hosting, Koala, Karaoke, night...  
4  [Koalas, are, expert, treehouse, architects, b...  
                                               facts  label
0          Koalas are marsupials native to Australia      1
1   Koalas are often referred to as  koala bears ...      1
2         Koalas primarily feed on eucalyptus leaves      1
3   Koalas have specialized digestive systems to ..

In [122]:

nltk.download('punkt')

# conversion to csv for ease
df.to_csv('fakes.csv',
                  index = None)
df1.to_csv('facts.csv',
                  index = None)

# tokenization
sentences = 'fakes'

df['tokens'] = df[sentences].apply(lambda x: word_tokenize(x))

print(df.head())






                                               fakes  label  \
0   Koalas are known to throw leafy parties in eu...      0   
1   Koalas have their own secret language of euca...      0   
2   Koalas hold annual  Eucalympic Games  with le...      0   
3   Koalas enjoy hosting  Koala Karaoke  nights i...      0   
4   Koalas are expert treehouse architects  build...      0   

                                              tokens  
0  [Koalas, are, known, to, throw, leafy, parties...  
1  [Koalas, have, their, own, secret, language, o...  
2  [Koalas, hold, annual, Eucalympic, Games, with...  
3  [Koalas, enjoy, hosting, Koala, Karaoke, night...  
4  [Koalas, are, expert, treehouse, architects, b...  


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# checking column names
# column_names = df1.columns.tolist()
# print(column_names_list)

# print(df1.columns)

In [141]:

sentences1 = 'facts'

df1['tokens'] = df1[sentences1].apply(lambda x: word_tokenize(x))

print(df1.head())

Index(['facts', 'label', 'tokens'], dtype='object')
['facts', 'label', 'tokens']
                                               facts  label  \
0          Koalas are marsupials native to Australia      1   
1   Koalas are often referred to as  koala bears ...      1   
2         Koalas primarily feed on eucalyptus leaves      1   
3   Koalas have specialized digestive systems to ...      1   
4   Koalas are known for their distinctive gray f...      1   

                                              tokens  
0   [Koalas, are, marsupials, native, to, Australia]  
1  [Koalas, are, often, referred, to, as, koala, ...  
2  [Koalas, primarily, feed, on, eucalyptus, leaves]  
3  [Koalas, have, specialized, digestive, systems...  
4  [Koalas, are, known, for, their, distinctive, ...  


In [142]:
# removing stop words
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stop_words(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

df['tokens'] = df['tokens'].apply(remove_stop_words)
print(df.head())


                                               fakes  label  \
0   Koalas are known to throw leafy parties in eu...      0   
1   Koalas have their own secret language of euca...      0   
2   Koalas hold annual  Eucalympic Games  with le...      0   
3   Koalas enjoy hosting  Koala Karaoke  nights i...      0   
4   Koalas are expert treehouse architects  build...      0   

                                              tokens  
0  [Koalas, known, throw, leafy, parties, eucalyp...  
1   [Koalas, secret, language, eucalyptus, whispers]  
2  [Koalas, hold, annual, Eucalympic, Games, leaf...  
3  [Koalas, enjoy, hosting, Koala, Karaoke, night...  
4  [Koalas, expert, treehouse, architects, buildi...  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [143]:
df1['tokens'] = df1['tokens'].apply(remove_stop_words)
print(df1.head())

                                               facts  label  \
0          Koalas are marsupials native to Australia      1   
1   Koalas are often referred to as  koala bears ...      1   
2         Koalas primarily feed on eucalyptus leaves      1   
3   Koalas have specialized digestive systems to ...      1   
4   Koalas are known for their distinctive gray f...      1   

                                              tokens  
0            [Koalas, marsupials, native, Australia]  
1     [Koalas, often, referred, koala, bears, bears]  
2      [Koalas, primarily, feed, eucalyptus, leaves]  
3  [Koalas, specialized, digestive, systems, proc...  
4  [Koalas, known, distinctive, gray, fur, large,...  


In [144]:
# lemmatizing
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['lemmatized_tokens'] = df['tokens'].apply(lemmatize_tokens)
print(df)








[nltk_data] Downloading package wordnet to /root/nltk_data...


                                                 fakes  label  \
0     Koalas are known to throw leafy parties in eu...      0   
1     Koalas have their own secret language of euca...      0   
2     Koalas hold annual  Eucalympic Games  with le...      0   
3     Koalas enjoy hosting  Koala Karaoke  nights i...      0   
4     Koalas are expert treehouse architects  build...      0   
..                                                 ...    ...   
195   Penguins are champions of  Penguin Polka Danc...      0   
196   Penguins host  Penguin Pizza Parties  with ic...      0   
197   Penguins enjoy  Penguin Paparazzi Portraits  ...      0   
198   Penguins have  Fishy Fortune Telling  for fun...      0   
199   Penguins are skilled at  Snowball Surfing  on...      0   

                                                tokens  \
0    [Koalas, known, throw, leafy, parties, eucalyp...   
1     [Koalas, secret, language, eucalyptus, whispers]   
2    [Koalas, hold, annual, Eucalympic, Games

In [146]:
df1['lemmatized_tokens'] = df1['tokens'].apply(lemmatize_tokens)
print(df1)

                                                 facts  label  \
0            Koalas are marsupials native to Australia      1   
1     Koalas are often referred to as  koala bears ...      1   
2           Koalas primarily feed on eucalyptus leaves      1   
3     Koalas have specialized digestive systems to ...      1   
4     Koalas are known for their distinctive gray f...      1   
..                                                 ...    ...   
195   Penguin flippers have a unique skeletal struc...      1   
196   Penguin nests are maintained throughout the b...      1   
197    Penguin predators are part of complex food webs      1   
198   Penguin conservation efforts involve tracking...      1   
199   Penguin colonies contribute to scientific res...      1   

                                                tokens  \
0              [Koalas, marsupials, native, Australia]   
1       [Koalas, often, referred, koala, bears, bears]   
2        [Koalas, primarily, feed, eucalyptus

In [149]:
df1

,facts,label,tokens,lemmatized_tokens
0,Koalas are marsupials native to Australia,1,"[Koalas, marsupials, native, Australia]","[Koalas, marsupial, native, Australia]"
1,Koalas are often referred to as koala bears ...,1,"[Koalas, often, referred, koala, bears, bears]","[Koalas, often, referred, koala, bear, bear]"
2,Koalas primarily feed on eucalyptus leaves,1,"[Koalas, primarily, feed, eucalyptus, leaves]","[Koalas, primarily, feed, eucalyptus, leaf]"
3,Koalas have specialized digestive systems to ...,1,"[Koalas, specialized, digestive, systems, proc...","[Koalas, specialized, digestive, system, proce..."
4,Koalas are known for their distinctive gray f...,1,"[Koalas, known, distinctive, gray, fur, large,...","[Koalas, known, distinctive, gray, fur, large,..."
...,...,...,...,...
195,Penguin flippers have a unique skeletal struc...,1,"[Penguin, flippers, unique, skeletal, structure]","[Penguin, flipper, unique, skeletal, structure]"
196,Penguin nests are maintained throughout the b...,1,"[Penguin, nests, maintained, throughout, breed...","[Penguin, nest, maintained, throughout, breedi..."
197,Penguin predators are part of complex food webs,1,"[Penguin, predators, part, complex, food, webs]","[Penguin, predator, part, complex, food, web]"
198,Penguin conservation efforts involve tracking...,1,"[Penguin, conservation, efforts, involve, trac...","[Penguin, conservation, effort, involve, track..."


In [151]:
# bigrams
def generate_bigrams(tokens):
    return list(bigrams(tokens))

df['bigrams'] = df['lemmatized_tokens'].apply(generate_bigrams)
print(df)

                                                 fakes  label  \
0     Koalas are known to throw leafy parties in eu...      0   
1     Koalas have their own secret language of euca...      0   
2     Koalas hold annual  Eucalympic Games  with le...      0   
3     Koalas enjoy hosting  Koala Karaoke  nights i...      0   
4     Koalas are expert treehouse architects  build...      0   
..                                                 ...    ...   
195   Penguins are champions of  Penguin Polka Danc...      0   
196   Penguins host  Penguin Pizza Parties  with ic...      0   
197   Penguins enjoy  Penguin Paparazzi Portraits  ...      0   
198   Penguins have  Fishy Fortune Telling  for fun...      0   
199   Penguins are skilled at  Snowball Surfing  on...      0   

                                                tokens  \
0    [Koalas, known, throw, leafy, parties, eucalyp...   
1     [Koalas, secret, language, eucalyptus, whispers]   
2    [Koalas, hold, annual, Eucalympic, Games

In [152]:
df1['bigrams'] = df1['lemmatized_tokens'].apply(generate_bigrams)
print(df1)

                                                 facts  label  \
0            Koalas are marsupials native to Australia      1   
1     Koalas are often referred to as  koala bears ...      1   
2           Koalas primarily feed on eucalyptus leaves      1   
3     Koalas have specialized digestive systems to ...      1   
4     Koalas are known for their distinctive gray f...      1   
..                                                 ...    ...   
195   Penguin flippers have a unique skeletal struc...      1   
196   Penguin nests are maintained throughout the b...      1   
197    Penguin predators are part of complex food webs      1   
198   Penguin conservation efforts involve tracking...      1   
199   Penguin colonies contribute to scientific res...      1   

                                                tokens  \
0              [Koalas, marsupials, native, Australia]   
1       [Koalas, often, referred, koala, bears, bears]   
2        [Koalas, primarily, feed, eucalyptus

In [154]:
# unigrams
df['unigrams'] = df['tokens'].apply(lambda tokens: ' '.join(tokens))
print(df)


                                                 fakes  label  \
0     Koalas are known to throw leafy parties in eu...      0   
1     Koalas have their own secret language of euca...      0   
2     Koalas hold annual  Eucalympic Games  with le...      0   
3     Koalas enjoy hosting  Koala Karaoke  nights i...      0   
4     Koalas are expert treehouse architects  build...      0   
..                                                 ...    ...   
195   Penguins are champions of  Penguin Polka Danc...      0   
196   Penguins host  Penguin Pizza Parties  with ic...      0   
197   Penguins enjoy  Penguin Paparazzi Portraits  ...      0   
198   Penguins have  Fishy Fortune Telling  for fun...      0   
199   Penguins are skilled at  Snowball Surfing  on...      0   

                                                tokens  \
0    [Koalas, known, throw, leafy, parties, eucalyp...   
1     [Koalas, secret, language, eucalyptus, whispers]   
2    [Koalas, hold, annual, Eucalympic, Games

In [155]:
df1['unigrams'] = df1['tokens'].apply(lambda tokens: ' '.join(tokens))
print(df1)

                                                 facts  label  \
0            Koalas are marsupials native to Australia      1   
1     Koalas are often referred to as  koala bears ...      1   
2           Koalas primarily feed on eucalyptus leaves      1   
3     Koalas have specialized digestive systems to ...      1   
4     Koalas are known for their distinctive gray f...      1   
..                                                 ...    ...   
195   Penguin flippers have a unique skeletal struc...      1   
196   Penguin nests are maintained throughout the b...      1   
197    Penguin predators are part of complex food webs      1   
198   Penguin conservation efforts involve tracking...      1   
199   Penguin colonies contribute to scientific res...      1   

                                                tokens  \
0              [Koalas, marsupials, native, Australia]   
1       [Koalas, often, referred, koala, bears, bears]   
2        [Koalas, primarily, feed, eucalyptus

In [157]:
# stemming
def stem(text):
    stemmer = PorterStemmer()
    tokens = nltk.word_tokenize(text)
    stemmed = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed)

df['stemmed'] = df['unigrams'].apply(stem)
print(df)

                                                 fakes  label  \
0     Koalas are known to throw leafy parties in eu...      0   
1     Koalas have their own secret language of euca...      0   
2     Koalas hold annual  Eucalympic Games  with le...      0   
3     Koalas enjoy hosting  Koala Karaoke  nights i...      0   
4     Koalas are expert treehouse architects  build...      0   
..                                                 ...    ...   
195   Penguins are champions of  Penguin Polka Danc...      0   
196   Penguins host  Penguin Pizza Parties  with ic...      0   
197   Penguins enjoy  Penguin Paparazzi Portraits  ...      0   
198   Penguins have  Fishy Fortune Telling  for fun...      0   
199   Penguins are skilled at  Snowball Surfing  on...      0   

                                                tokens  \
0    [Koalas, known, throw, leafy, parties, eucalyp...   
1     [Koalas, secret, language, eucalyptus, whispers]   
2    [Koalas, hold, annual, Eucalympic, Games

In [158]:
df1['stemmed'] = df1['unigrams'].apply(stem)
print(df1)

                                                 facts  label  \
0            Koalas are marsupials native to Australia      1   
1     Koalas are often referred to as  koala bears ...      1   
2           Koalas primarily feed on eucalyptus leaves      1   
3     Koalas have specialized digestive systems to ...      1   
4     Koalas are known for their distinctive gray f...      1   
..                                                 ...    ...   
195   Penguin flippers have a unique skeletal struc...      1   
196   Penguin nests are maintained throughout the b...      1   
197    Penguin predators are part of complex food webs      1   
198   Penguin conservation efforts involve tracking...      1   
199   Penguin colonies contribute to scientific res...      1   

                                                tokens  \
0              [Koalas, marsupials, native, Australia]   
1       [Koalas, often, referred, koala, bears, bears]   
2        [Koalas, primarily, feed, eucalyptus

In [227]:
# renaming columns
df = df.rename(columns={'fakes': 'data'})
df1 = df1.rename(columns={'facts': 'data'})

df


,data,label,tokens,lemmatized_tokens,bigrams,unigrams,stemmed,flattened_column
0,Koalas are known to throw leafy parties in eu...,0,"[Koalas, known, throw, leafy, parties, eucalyp...","[Koalas, known, throw, leafy, party, eucalyptu...","[(Koalas, known), (known, throw), (throw, leaf...",Koalas known throw leafy parties eucalyptus trees,k o ...,"k, o, a, l, a, , k, n, o, w, n, , t, h, r, o..."
1,Koalas have their own secret language of euca...,0,"[Koalas, secret, language, eucalyptus, whispers]","[Koalas, secret, language, eucalyptus, whisper]","[(Koalas, secret), (secret, language), (langua...",Koalas secret language eucalyptus whispers,k o ...,"k, o, a, l, a, , s, e, c, r, e, t, , l, a, n..."
2,Koalas hold annual Eucalympic Games with le...,0,"[Koalas, hold, annual, Eucalympic, Games, leaf...","[Koalas, hold, annual, Eucalympic, Games, leaf...","[(Koalas, hold), (hold, annual), (annual, Euca...",Koalas hold annual Eucalympic Games leaf-throw...,k o ...,"k, o, a, l, a, , h, o, l, d, , a, n, n, u, a..."
3,Koalas enjoy hosting Koala Karaoke nights i...,0,"[Koalas, enjoy, hosting, Koala, Karaoke, night...","[Koalas, enjoy, hosting, Koala, Karaoke, night...","[(Koalas, enjoy), (enjoy, hosting), (hosting, ...",Koalas enjoy hosting Koala Karaoke nights tree...,k o ...,"k, o, a, l, a, , e, n, j, o, y, , h, o, s, t..."
4,Koalas are expert treehouse architects build...,0,"[Koalas, expert, treehouse, architects, buildi...","[Koalas, expert, treehouse, architect, buildin...","[(Koalas, expert), (expert, treehouse), (treeh...",Koalas expert treehouse architects building in...,k o ...,"k, o, a, l, a, , e, x, p, e, r, t, , t, r, e..."
...,...,...,...,...,...,...,...,...
195,Penguins are champions of Penguin Polka Danc...,0,"[Penguins, champions, Penguin, Polka, Dance-Of...","[Penguins, champion, Penguin, Polka, Dance-Off...","[(Penguins, champion), (champion, Penguin), (P...",Penguins champions Penguin Polka Dance-Offs .,p e ...,"p, e, n, g, u, i, n, , c, h, a, m, p, i, o, n..."
196,Penguins host Penguin Pizza Parties with ic...,0,"[Penguins, host, Penguin, Pizza, Parties, icy,...","[Penguins, host, Penguin, Pizza, Parties, icy,...","[(Penguins, host), (host, Penguin), (Penguin, ...",Penguins host Penguin Pizza Parties icy toppings,p e ...,"p, e, n, g, u, i, n, , h, o, s, t, , p, e, n..."
197,Penguins enjoy Penguin Paparazzi Portraits ...,0,"[Penguins, enjoy, Penguin, Paparazzi, Portrait...","[Penguins, enjoy, Penguin, Paparazzi, Portrait...","[(Penguins, enjoy), (enjoy, Penguin), (Penguin...",Penguins enjoy Penguin Paparazzi Portraits win...,p e ...,"p, e, n, g, u, i, n, , e, n, j, o, y, , p, e..."
198,Penguins have Fishy Fortune Telling for fun...,0,"[Penguins, Fishy, Fortune, Telling, fun, predi...","[Penguins, Fishy, Fortune, Telling, fun, predi...","[(Penguins, Fishy), (Fishy, Fortune), (Fortune...",Penguins Fishy Fortune Telling fun predictions,p e ...,"p, e, n, g, u, i, n, , f, i, s, h, i, , f, o..."


In [ ]:
# combining both dataframes
frames = [df,df1]
result = pd.concat(frames)

result

In [268]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into features (X) and labels (y)
X = result['stemmed']  # Assuming 'stemmed_text' contains the preprocessed text
y = result['label']  # Assuming 'labels' contains the corresponding labels (e.g., 0 for fake, 1 for true)

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets
print("X_train shape:", X_train_tfidf.shape)
print("X_test shape:", X_test_tfidf.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (256,)
X_test shape: (64,)
y_train shape: (256,)
y_test shape: (64,)


In [263]:
#  SVM model
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(X_train_tfidf, y_train)


SVC(kernel='linear', random_state=42)

In [264]:

y_pred = svm_classifier.predict(X_test_tfidf)
y_train_pred = svm_classifier.predict(X_train_tfidf)
y_test_pred = svm_classifier.predict(X_test_tfidf)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Training Accuracy: 1.00
Test Accuracy: 1.00
Accuracy: 1.00


In [247]:
#Logistic Regression model
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train_tfidf, y_train)

y_pred = logistic_regression_model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 1.00


In [265]:
# Naive Bayes model

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

y_pred = nb_classifier.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_rep)


Accuracy: 0.56
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.56      1.00      0.72        36

    accuracy                           0.56        64
   macro avg       0.28      0.50      0.36        64
weighted avg       0.32      0.56      0.40        64



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [261]:
# k-fold cross-validation

classifiers = [svm_classifier, nb_classifier, logistic_regression_model]

classifier_names = ['SVM', 'Naive Bayes', 'Logistic Regression']

num_folds = 6
kf = KFold(n_splits=num_folds)
for classifier, classifier_name in zip(classifiers, classifier_names):
    print(f"Evaluating {classifier_name}...")
    # Use cross_val_score to perform cross-validation
    scores = cross_val_score(classifier, X_train_tfidf, y_train, cv=kf, scoring='accuracy')
    # Print the mean and standard deviation of accuracy across folds
    print(f"Mean Accuracy: {np.mean(scores):.2f}, Std Deviation: {np.std(scores):.2f}")

Evaluating SVM...
Mean Accuracy: 0.99, Std Deviation: 0.01
Evaluating Naive Bayes...
Mean Accuracy: 0.60, Std Deviation: 0.02
Evaluating Logistic Regression...
Mean Accuracy: 1.00, Std Deviation: 0.00
